In [69]:
import agentpy as ap
import matplotlib.pyplot as plt
import seaborn as sns
import IPython 
import random
import numpy as np

In [70]:
class RoomModel(ap.Model): #
    def robotSpawnSetup(numbRobots):
        coordList = ()
        for i in range(numbRobots):
            coordList.append((1,1))
        return coordList
    def setup(self):
        
        #Creamos agentes limpiadores 
        n_robots = int(self.p['Number of robots'])
        robot =self.robot_agents=ap.AgentList(self,n_robots)
        robot.moves=0
        robot.condition=2
        # Creamos agentes espacio en cuarto
        n_dirt = int(self.p['Dirty room density'] *self.p.N*self.p.M)
        dirt = self.dirt_agents = ap.AgentList(self, n_dirt)
        dirt.condition=1
        # Create grid (rooms)
        self.room = ap.Grid(self, [self.p.N,self.p.M], track_empty=True)
        self.room.add_agents(dirt, random=True, empty=True)
        self.room.add_agents(robot,((1,1),(1,1),(1,1)))

        


    def step(self):
        n_moves = np.sum(np.array(self.robot_agents.moves))
        self.record('Moves',n_moves)
        #Select robots 
        cleaners = self.robot_agents.select(self.robot_agents.condition == 2)
        
        for i in cleaners:
            self.room.move_by(i,(random.randint(-1,2),random.randint(-1,2)))
            for neighbor in self.room.neighbors(i,0):
                if neighbor.condition == 1: #dirty neighbor
                    neighbor.condition = 0 #cleaned
        self.robot_agents.moves +=1

        # Select dirty spaces
        dirty_spaces = self.dirt_agents.select(self.dirt_agents.condition == 1)
        self.record('Dirty cells', dirty_spaces)
        # Stop simulation if no fire is left
        if len(dirty_spaces) == 0:
            self.stop()
        
    def end(self):

        # Document a measure at the end of the simulation
        clean_spacesFinal = len(self.dirt_agents.select(self.dirt_agents.condition == 0))
        self.report('Percentage of clean spaces',
                    clean_spacesFinal / len(self.dirt_agents))
        self.report('Moves', np.sum(np.array(self.robot_agents.moves)))
        self.report('Time',self.t)
        

In [71]:
parameters = {
    'Number of robots': 3,
    'N': 15,
    'M': 15,
    'Dirty room density': 0.6, # Percentage of grid covered by trees
    'steps': 100,
}


In [72]:
def animation_plot(model, ax):
    attr_grid = model.room.attr_grid('condition')
    color_dict = {0:'#7FC97F', 1:'#d62c2c', 2:'#0000ff', None:'#d5e5d5'} #verde limpio y rojo sucio
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulation of a room cleaning\n"
                 f"Time-step: {model.t}, rooms left: "
                 f"{len(model.dirt_agents.select(model.dirt_agents.condition == 0))}")

fig, ax = plt.subplots()
model = RoomModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))